In [ ]:
import sympy as sp
from sympy import sin, cos, pi
from sympy.interactive import printing
import pickle
import numpy as np
import scipy as sc
import scipy.interpolate
from scipy.integrate import odeint
import matplotlib.pyplot as plt

import symbtools as st
import symbtools.modeltools as mt
import symbtools.noncommutativetools as nct

mathematisches Pendel der Länge $l$ 

$\varphi=0$ entspricht der oberen Ruhelage

In [ ]:
t = sp.Symbol('t')
Np = 1
Nq = 1
n = Np + Nq
pp = st.symb_vector("p1:{0}".format(Np+1))
qq = st.symb_vector("q1:{0}".format(Nq+1))
aa = st.symb_vector("a1:{0}".format(Nq+1))
ww = st.symb_vector("w1:{0}".format(Nq+1))

ttheta = st.row_stack(pp, qq) ##:T
tthetad = st.time_deriv(ttheta, ttheta) ##:T
tthetadd = st.time_deriv(ttheta, ttheta, order=2) ##:T
st.make_global(ttheta, tthetad)

In [ ]:
params = sp.symbols('m1, m2, l, g') # m1 wagen, m2 pole
st.make_global(params)

tau1 = sp.Symbol("tau1")

In [ ]:
#Einheitsvektoren
ex = sp.Matrix([1,0])
ey = sp.Matrix([0,1])


# Koordinaten der Schwerpunkte und Gelenke
S1 = ex*q1 # Schwerpunkt Wagen
G2 = S1 # Pendel-Gelenk

# Schwerpunkt des Pendels (Pendel zeigt für kleine Winkel nach #! OBEN)
S2 = G2 + sp.Matrix([l * sin(p1), l*cos(p1)]) 

# Zeitableitungen der Schwerpunktskoordinaten
Sd1, Sd2  = st.col_split(st.time_deriv(st.col_stack(S1, S2), ttheta)) ##

In [ ]:
# Energie
T_rot = 0 # (Punktmassenmodell)
T_trans = ( m1*Sd1.T*Sd1  +  m2*Sd2.T*Sd2 )/2

T = T_rot + T_trans[0]

V = m2*g*S2[1]

In [ ]:
mod = mt.generate_symbolic_model(T, V, ttheta, [0, tau1])

In [ ]:
mod.MM.simplify()
mod.MM

In [ ]:
mod.eqns

In [ ]:
mod.calc_state_eq(simplify=True)
sp.simplify(mod.state_eq)

In [ ]:
repr(sp.simplify(mod.state_eq))


In [ ]:
sp.simplify(mod.state_eq.subs(l, 4/3*l))

In [ ]:
mod.calc_coll_part_lin_state_eq(simplify=True)

In [ ]:
xx = mod.x ##:T
f = mod.ff ##:
G = mod.gg 
g1, = st.col_split(G) ##:
xx

In [ ]:
f

In [ ]:
G

In [ ]:
eqp1 = sp.Matrix([0, 0, 0, 0 ]) ##:T
eqp2 = sp.Matrix([sp.pi, 0, 0, 0]) ##:T

# Probe:
f.subz(xx, eqp1) ##:T
f.subz(xx, eqp2) ##:T

In [ ]:
# Parameterwerte
parameter_values = [(g, 9.81), (l, .5)]
replm =  parameter_values + list(zip(xx, eqp1)) ##
replm

In [ ]:
A1 = f.jacobian(xx).subs(replm) ##
A1

In [ ]:
b1 = G.subs(replm) ##
b1

In [ ]:
a = 1.2
b = 3.4

poles = np.r_[-1.5+a*1j, -1.5-a*1j, -1.3 + b*1j, -1.3 - b*1j] ##:
# poles = np.linspace(-5, -8, 4)
# k1 = st.siso_place(A1, b1, poles)
# print(k1)
# A1-b1@k1
import control as ctrl
F = ctrl.place(np.array(A1, dtype=float), np.array(b1, dtype=float), poles)
print("F", F)
print("F in env order of x, xdot, phi, phidot:")
F_env = [F[0,1], F[0,3], F[0,0], F[0,2]]
print(F_env)
np.linalg.eigvals(np.array(A1, dtype=float) - np.array(b1, dtype=float) @ F)

In [ ]:
k1 = st.siso_place(A1, b1, poles)##:T

controller1_expr1 = k1.T*mod.xx
controller1_func_intern = st.expr_to_func(mod.xx, k1.T*mod.xx)

def controller1_func(xx, t):
    # Zeit t ignorieren
    return controller1_func_intern(*xx)

k1

In [ ]:
# Probe
A = A1 + b1*k1.T
A.eigenvals(rational=False)

In [ ]:
# Zeit-Array und Anfangswerte
tt = np.arange(0, 10, 1e-3)
xx0 = [0, 0, 0.1, 0] ##

#Erstellung des Simulationsmodells
sm = st.SimulationModel(mod.ff, mod.gg, mod.xx, model_parameters=parameter_values)
rhs1 = sm.create_simfunction(controller_function=controller1_func)

# Durchführung der Simulation
res = odeint(rhs1, xx0, tt)
X1, X2, X3, X4 = res.T
aa1 = controller1_func_intern(X1, X2, X3, X4)

In [ ]:
T_fin = 6
plt.figure(figsize=(15,4))
plt.subplot(1, 3, 1)
plt.plot(tt, X1, label="phi")
plt.legend(loc='best', fontsize=16)
plt.subplot(1, 3, 2)
plt.plot(tt, X2, label="x")
plt.legend(loc='best', fontsize=16)
plt.subplot(1, 3, 3)
plt.plot(tt, aa1, label="u")
plt.vlines([T_fin], -20, 20, color='0.8', label=r'Tend')
plt.legend(loc='best', fontsize=16)

Simulation

In [ ]:
from scipy.integrate import solve_ivp
def rhs(t, state):
    x, x_dot, theta, theta_dot = state
    x1, x2, x3, x4 = x, theta, x_dot, theta_dot # change order
    g = 9.8
    l = 0.5
    m1 = 1.0
    m2 = 0.1
    F = np.array([[-23.42598624,  -2.49197248,  -4.05749235,  -2.51498471]])
    u1 = - F @ np.array([x1, x2, x3, x4])
    dx1_dt = x3
    dx2_dt = x4
    dx3_dt = (-g*m2*np.sin(2*x2)/2 + l*m2*theta_dot**2*np.sin(x2) + u1)/(m1 + m2*np.sin(x2)**2)
    dx4_dt = (g*(m1 + m2)*np.sin(x2) - (l*m2*theta_dot**2*np.sin(x2) + u1)*np.cos(x2))/\
        (l*(m1 + m2*np.sin(x2)**2))
    
    return [dx1_dt, dx3_dt, dx2_dt, dx4_dt] # change order back

tend = 10
tt = np.linspace(0, tend, 2000)
xx0 = [0, 0, 0.01, 0]
s = solve_ivp(rhs, (0, tend), xx0, t_eval=tt)
labels = ["x", "xdot", "phi", "phidot"]
for i in range(4):
    plt.plot(s.t, s.y[i], label=labels[i])
plt.legend()
plt.show()